In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

In [ ]:
from thesis.config import load_config, GaussianSplattingSettings

path = "configs/single_frame.yml"
config = load_config(path)

In [ ]:
from thesis.gaussian_splatting.initialize_splats import random_initialization

random_initialization(10)

In [ ]:
from torch_geometric.nn.pool import knn

means = torch.tensor([[0, 0], [1, 0], [0, 2], [1, 1]], dtype=torch.float)
sender, receiver = knn(means, means, k=4)
sender = sender.reshape(-1, 4)[:, 1:].reshape(-1)  # Remove self
receiver = receiver.reshape(-1, 4)[:, 1:].reshape(-1)